In [1]:

from dotenv import load_dotenv
import duckdb
from pathlib import Path

load_dotenv()

True

In [2]:
base_path = Path().cwd().parent
source_path = base_path / Path('silver/anilist/link/link-2024-1-25.parquet')
target_path = base_path / Path('gold/anilist/dim-link.parquet')

In [3]:
tb = duckdb.read_parquet(str(source_path))

In [4]:
tb.shape

(591, 7)

In [5]:
tb_dim = duckdb.sql("""
WITH tb_filter_col AS (
    SELECT
        id AS url_id
        , url
        , site
        , type AS site_type
        , siteId AS site_id
        , language
    FROM tb
), dedupe AS (
    SELECT
        url_id
        , ANY_VALUE(url) AS url
        , ANY_VALUE(site) AS site
        , ANY_VALUE(site_type) AS site_type
        , ANY_VALUE(site_id) AS site_id
        , ANY_VALUE(language) AS language
    FROM tb_filter_col
    GROUP BY (url_id )
)
SELECT *
FROM dedupe
""")

In [6]:
tb_dim.shape

(589, 6)

In [7]:
tb_dim.to_parquet(str(target_path))